In [ ]:
import pandas as pd
import re
import os
from fuzzywuzzy import fuzz
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from tqdm import tqdm
from datetime import datetime

In [ ]:
# === CONFIG ===
STUDY_FILE = './out/HDP00002_HELPForNOWS_INFORMNOW_DataDictionary_REDCap.vlmd_2025-07-22.xlsx'
STUDY_SHEET = 'EnhancedDD'
ENCODING_COLUMN = 'enumLabels'
FIELD_LABEL_COLUMN = 'description'
CDE_FILE = './KnowledgeBase/Compiled_CORE_CDEs list_English_one sheet_as of 2025-01-28.xlsx'
OUTPUT_DIR = 'out'

In [ ]:
# Configurable thresholds
CONFIDENCE_THRESHOLDS = {
    'high': 80,
    'medium': 51,
    'minimum_score': 30  # Don't show matches below this threshold
}

# Weighting for different text components
TEXT_WEIGHT = 0.4
ENCODING_WEIGHT = 0.6

In [ ]:
def normalize_string(s):
    """Normalize string by converting to lowercase, handling NaN or float, and keeping spaces."""
    if isinstance(s, str):
        s = re.sub(r'[^a-zA-Z0-9\s=]', '', s.lower())  # Keep letters, numbers, spaces, and equal signs
        s = re.sub(r'\s+', ' ', s)  # Collapse multiple spaces into one
        return s.strip()
    else:
        return ''

def similarity_score(str1, str2):
    """
    Calculate token-based fuzzy similarity between two strings.
    100% means an exact match, 0% means completely different.
    """
    return fuzz.token_set_ratio(str1, str2)

def enhanced_similarity_score(study_text, study_encoding, cde_text, cde_encoding):
    """
    Enhanced similarity scoring that weights encodings and text differently.
    Encodings are often more standardized, so they get higher weight.
    """
    # Handle cases where components might be empty
    if not study_encoding and not cde_encoding:
        return similarity_score(study_text, cde_text)
    if not study_text and not cde_text:
        return similarity_score(study_encoding, cde_encoding)
    
    text_score = similarity_score(study_text, cde_text) if study_text and cde_text else 0
    encoding_score = similarity_score(study_encoding, cde_encoding) if study_encoding and cde_encoding else 0
    
    # Weighted average
    return (text_score * TEXT_WEIGHT) + (encoding_score * ENCODING_WEIGHT)

def print_matching_summary(final_df):
    """Print summary statistics of the matching process."""
    processed_df = final_df[final_df['Best Match CDE Name'].notna()]
    total_processed = len(processed_df)
    
    if total_processed == 0:
        print("📊 No matches found.")
        return
    
    high_conf = len(processed_df[processed_df['Best Match Score'] >= CONFIDENCE_THRESHOLDS['high']])
    medium_conf = len(processed_df[
        (processed_df['Best Match Score'] >= CONFIDENCE_THRESHOLDS['medium']) & 
        (processed_df['Best Match Score'] < CONFIDENCE_THRESHOLDS['high'])
    ])
    low_conf = len(processed_df[processed_df['Best Match Score'] < CONFIDENCE_THRESHOLDS['medium']])
    
    print(f"\n📊 Matching Summary:")
    print(f"   Total variables processed: {total_processed}")
    print(f"   🟢 High confidence matches (≥{CONFIDENCE_THRESHOLDS['high']}%): {high_conf} ({high_conf/total_processed*100:.1f}%)")
    print(f"   🟧 Medium confidence matches ({CONFIDENCE_THRESHOLDS['medium']}-{CONFIDENCE_THRESHOLDS['high']-1}%): {medium_conf} ({medium_conf/total_processed*100:.1f}%)")
    print(f"   🟥 Low confidence matches (<{CONFIDENCE_THRESHOLDS['medium']}%): {low_conf} ({low_conf/total_processed*100:.1f}%)")

def compare_encodings(
    study_file,
    encoding_column='encodings',
    field_label_column='field_label',
    cde_file='./KnowledgeBase/Compiled_CORE_CDEs list_English_one sheet_as of 2025-01-28.xlsx',
    study_sheet='Sheet1',
    dry_run=False
):
    """
    Compare study data dictionary encodings and field labels with HEAL CDE encodings using fuzzy token-based similarity.
    
    Parameters:
    - study_file: Path to the study data dictionary file (.xlsx or .csv).
    - encoding_column: Name of the column containing encodings in the study file.
    - field_label_column: Name of the column containing field labels in the study file.
    - cde_file: Path to the HEAL CDE knowledge base file (.xlsx).
    - study_sheet: Name of the sheet in the study file to process (default is 'Sheet1').
    - dry_run: If True, only preview the process without actually running matches.
    
    Returns:
    - str: Path to the output file with CDE matches.
    """
    
    # Ensure output directory exists
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # Load study data
    print("📂 Loading study data...")
    if study_file.endswith('.xlsx'):
        full_study_df = pd.read_excel(study_file, sheet_name=study_sheet)
    else:
        full_study_df = pd.read_csv(study_file)

    # Filter out 'No CRF match rows' - only process rows that have confirmed CRF matches
    if 'HEAL Core CRF Match' in full_study_df.columns:
        skipped_df = full_study_df[full_study_df['HEAL Core CRF Match'] == 'No CRF match'].copy()
        study_df = full_study_df[full_study_df['HEAL Core CRF Match'] != 'No CRF match'].copy()
        print(f"✅ Processing {len(study_df)} rows; skipping {len(skipped_df)} rows (No CRF match).")
    else:
        skipped_df = pd.DataFrame()
        study_df = full_study_df.copy()
        print(f"✅ Processing all {len(study_df)} rows (no CRF match column found).")

    if dry_run:
        print(f"🔍 DRY RUN: Would process {len(study_df)} variables against HEAL CDEs")
        return None

    # Initialize new columns for CDE matches
    new_cols = [
        'Best Match CDE Name', 'Best Match Score', 'Best Match CRF Name',
        'Potential Match 2 - CDE Name', 'Potential Match 2 - Score', 'Potential Match 2 - CRF Name',
        'Potential Match 3 - CDE Name', 'Potential Match 3 - Score', 'Potential Match 3 - CRF Name'
    ]
    for col in new_cols:
        study_df[col] = None
    
    # Normalize study encodings and field labels for comparison
    print("🔧 Normalizing study variables...")
    study_df['Normalized Text'] = study_df[field_label_column].apply(normalize_string)
    study_df['Normalized Encoding'] = study_df[encoding_column].apply(normalize_string)
    study_df['Normalized Combined'] = study_df.apply(
        lambda row: normalize_string(str(row[encoding_column]) + " | " + str(row[field_label_column]))
        if pd.notna(row[encoding_column]) and pd.notna(row[field_label_column]) else '',
        axis=1
    )
    
    # Load and normalize HEAL CDE encodings
    print("📚 Loading HEAL CDE database...")
    cde_df = pd.read_excel(cde_file, sheet_name='ALL')
    cde_df = cde_df.dropna(subset=['PV Description', 'Additional Notes (Question Text)'])
    
    # Normalize CDE components separately for enhanced matching
    cde_df['Normalized Text'] = cde_df['Additional Notes (Question Text)'].apply(normalize_string)
    cde_df['Normalized Encoding'] = cde_df['PV Description'].apply(normalize_string)
    cde_df['Normalized Combined'] = (
        cde_df['Additional Notes (Question Text)'].fillna('') + " | " + cde_df['PV Description'].fillna('')
    ).apply(normalize_string)
    
    print(f"✅ Loaded {len(cde_df)} HEAL CDEs for comparison.")
    
    # Track low-confidence matches for analysis
    low_confidence_matches = []
    
    # Run comparisons only on filtered rows (those with confirmed CRF matches)
    print("🔍 Running enhanced fuzzy matching against HEAL CDEs...")
    
    for idx, row in tqdm(study_df.iterrows(), total=len(study_df), desc="Matching variables", unit="vars"):
        if row['Normalized Combined'] == '':
            continue

        best_match = None
        best_score = 0
        best_crf_name = None
        potential_matches = []

        # Compare against all HEAL CDEs using enhanced scoring
        for _, cde_row in cde_df.iterrows():
            # Use enhanced similarity that weights text and encodings differently
            score_percentage = enhanced_similarity_score(
                row['Normalized Text'], 
                row['Normalized Encoding'],
                cde_row['Normalized Text'], 
                cde_row['Normalized Encoding']
            )
            
            # Only consider matches above minimum threshold
            if score_percentage >= CONFIDENCE_THRESHOLDS['minimum_score']:
                if score_percentage > best_score:
                    best_match = cde_row['Variable Name']
                    best_score = score_percentage
                    best_crf_name = cde_row['CRF Name']
                potential_matches.append((cde_row['Variable Name'], score_percentage, cde_row['CRF Name']))

        # Sort potential matches by score and remove duplicates
        potential_matches.sort(key=lambda x: x[1], reverse=True)
        unique_matches, seen = [], set()
        for name, score, crf in potential_matches:
            if name not in seen:
                unique_matches.append((name, score, crf))
                seen.add(name)
        if best_match:
            unique_matches = [m for m in unique_matches if m[0] != best_match]

        # Save best match and top 2 alternatives
        if best_match:
            study_df.at[idx, 'Best Match CDE Name'] = best_match
            study_df.at[idx, 'Best Match Score'] = round(best_score, 1)
            study_df.at[idx, 'Best Match CRF Name'] = best_crf_name
            
            # Track low-confidence matches for analysis
            if best_score < CONFIDENCE_THRESHOLDS['high']:
                low_confidence_matches.append({
                    'Row': idx + 2,  # Excel row number
                    'Study_Variable': row.get('field_name', 'Unknown'),
                    'Study_Text': row['Normalized Combined'][:100] + '...' if len(row['Normalized Combined']) > 100 else row['Normalized Combined'],
                    'Best_CDE': best_match,
                    'Score': round(best_score, 1),
                    'CRF_Name': best_crf_name
                })
                
        for i, (match_name, match_score, crf_name) in enumerate(unique_matches[:2], start=2):
            study_df.at[idx, f'Potential Match {i} - CDE Name'] = match_name
            study_df.at[idx, f'Potential Match {i} - Score'] = round(match_score, 1)
            study_df.at[idx, f'Potential Match {i} - CRF Name'] = crf_name

    # Create low-confidence matches DataFrame for analysis
    low_conf_df = pd.DataFrame(low_confidence_matches) if low_confidence_matches else pd.DataFrame()

    # Merge skipped rows back with empty match columns
    for col in new_cols:
        if col not in skipped_df.columns:
            skipped_df[col] = None
    if 'Normalized Combined' not in skipped_df.columns:
        skipped_df['Normalized Combined'] = None

    final_df = pd.concat([study_df, skipped_df], ignore_index=True)

    # Save results with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_base = os.path.basename(study_file).rsplit('.', 1)[0]
    output_file = f"{OUTPUT_DIR}/{output_base}_vlmd_cdesearch_{timestamp}.xlsx"
    
    # Write multiple sheets
    with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
        final_df.to_excel(writer, sheet_name='VLMD_Results', index=False)
        
        if not low_conf_df.empty:
            low_conf_df.to_excel(writer, sheet_name='Low_Confidence_Analysis', index=False)
            print(f"📝 {len(low_conf_df)} low-confidence matches saved for analysis")
    
    print(f"💾 CDE matching complete. Results saved to {output_file}")
    
    # Print summary statistics
    print_matching_summary(final_df)

    return output_file

def apply_color_coding(output_file):
    """
    Apply color coding to the Excel file based on confidence levels:
    - Green (≥80): High confidence
    - Orange (51-79): Medium confidence  
    - Red (≤50): Low confidence
    """
    print("\n🎨 Applying color coding and confidence levels...")
    
    wb = load_workbook(output_file)
    
    # Work with the main results sheet
    ws = wb['VLMD_Results'] if 'VLMD_Results' in wb.sheetnames else wb.active

    # Find "Best Match Score" column
    best_match_score_col = None
    for idx, cell in enumerate(ws[1], start=1):
        if cell.value == "Best Match Score":
            best_match_score_col = idx
            break

    if best_match_score_col:
        # Insert Confidence Level column
        confidence_col_idx = best_match_score_col + 1
        ws.insert_cols(confidence_col_idx)
        ws.cell(row=1, column=confidence_col_idx).value = "Confidence Level"

        # Apply color coding and confidence levels
        for row_idx, row in enumerate(ws.iter_rows(min_row=2, min_col=best_match_score_col, max_col=best_match_score_col), start=2):
            for cell in row:
                score = cell.value
                confidence_cell = ws.cell(row=row_idx, column=confidence_col_idx)
                if score is not None:
                    if score >= CONFIDENCE_THRESHOLDS['high']:
                        confidence_cell.value = "High confidence"
                        cell.fill = PatternFill(start_color="C6EFCE", end_color="C6EFCE", fill_type="solid")  # Green
                    elif CONFIDENCE_THRESHOLDS['medium'] <= score < CONFIDENCE_THRESHOLDS['high']:
                        confidence_cell.value = "Medium confidence"
                        cell.fill = PatternFill(start_color="FFEB9C", end_color="FFEB9C", fill_type="solid")  # Orange
                    else:
                        confidence_cell.value = "Low confidence"
                        cell.fill = PatternFill(start_color="FFC7CE", end_color="FFC7CE", fill_type="solid")  # Red

    wb.save(output_file)
    print(f"✨ Color coding and confidence levels applied!")

def main(dry_run=False):
    """Main function to run the VLMD CDE matching process."""
    
    print("🚀 Starting HEAL CDE Variable Level Metadata Matching")
    print(f"📋 Configuration: High confidence ≥{CONFIDENCE_THRESHOLDS['high']}%, Medium ≥{CONFIDENCE_THRESHOLDS['medium']}%, Minimum threshold ≥{CONFIDENCE_THRESHOLDS['minimum_score']}%")
    
    # Run CDE matching
    output_file = compare_encodings(
        STUDY_FILE,
        encoding_column=ENCODING_COLUMN,
        field_label_column=FIELD_LABEL_COLUMN,
        study_sheet=STUDY_SHEET,
        cde_file=CDE_FILE,
        dry_run=dry_run
    )

    if not dry_run and output_file:
        # Apply color coding for easier review
        apply_color_coding(output_file)
        
        print(f"\n🎉 VLMD CDE matching complete!")
        print(f"📊 Results saved to: {output_file}")
        print(f"📋 Review the 'Low_Confidence_Analysis' sheet to improve future matching")

if __name__ == "__main__":
    # Set dry_run=True to preview without processing
    main(dry_run=False)

# HEAL CDE Fuzzy Matching Script

This script automates the matching of study data dictionaries to the HEAL Core Common Data Elements (CDEs), using smart fuzzy text comparison and a color-coded Excel output for easier review.

It is designed to help identify the **Best Match** and **Top 2 Potential Matches** for each study variable — even if the wording or encoding order is slightly different from the CDE standard.

---

## ✨ Key Features
- **Fuzzy Matching**: Uses token-based similarity (`Token Set Ratio`) to handle small typos and different word orders.
- **Normalized Comparisons**: Cleans and standardizes text for reliable matching.
- **Separate Output Folder**: All results are saved neatly into an `/out/` subfolder.
- **Color Coded Scores**:  
  - 🟩 **Green** for matches ≥ 80%  
  - 🟧 **Orange** for matches 51–79%  
  - 🟥 **Red** for matches ≤ 50%
- **No Duplicate Matches**: Ensures Best Match and Potential Matches are truly different.

---

## 🚀 How It Works

1. **Input**:
   - A study data dictionary (Excel `.xlsx` or CSV `.csv` file).
   - The master HEAL CDEs file (Excel file).

2. **Process**:
   - Normalize (clean) text by lowercasing, removing special characters, and preserving logical structures like equal signs.
   - Compare the study's "Encoding + Field Label" to the CDE's "PV Description + Question Text".
   - Select the best match and top two alternatives based on fuzzy matching scores.
   - Color-code the best match scores for easy visualization.

3. **Output**:
   - A new Excel file saved into `/out/`, with best matches listed and scores color-coded.

---

## 🛠️ Requirements

Install the following Python packages:

```bash
pip install pandas openpyxl fuzzywuzzy
```

---

## 📂 Folder Structure

```
/in/         # Input study files
/out/        # Output matched files (automatically created if not present)
/KnowledgeBase/ # Contains the HEAL Core CDE file
script.py    # Your main script
```

---

## 📋 Usage Example

```bash
python script.py
```

The output file will appear in the `/out/` folder and will be named something like:

```
SAMPLE_sprint_2020-12-16_vlmd_cdesearch.xlsx
```

---

## 🧠 Notes

- The script **requires** the correct columns to be named in the study file (e.g., `Choices, Calculations, OR Slider Labels` and `Field Label`).
- Only the **Best Match Score** column is color coded for quick review.
- Ensure your HEAL CDE master file contains the necessary columns: `PV Description` and `Additional Notes (Question Text)`.
